In [ ]:
import pandas as pd
import tzlocal

# Load your CSV
df = pd.read_csv('watch-history-enriched.csv')

In [ ]:
# Step 1: Parse datetime normally
df['watch_time_dt'] = pd.to_datetime(df['watch_time'], errors='coerce')

# Step 2: Detect system timezone
local_timezone = tzlocal.get_localzone()

# Step 3: Fix timezone correctly for each row
def fix_timezone(dt):
    if pd.isna(dt):
        return dt
    if dt.tzinfo is None:
        # Naive datetime, localize it
        return dt.tz_localize(local_timezone)
    else:
        # Already timezone-aware, just convert
        return dt.tz_convert(local_timezone)

df['watch_time_dt'] = df['watch_time_dt'].apply(fix_timezone)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame and 'duration_seconds' is filled correctly

# Step 1: Define duration in hours
df['duration_hours'] = df['duration_seconds'] / 3600

# Step 2: Bin durations into categories
bins = list(range(0, 13)) + [float('inf')]  # 0–1, 1–2, ..., 11–12, 12+
labels = [f"{i}-{i+1}" for i in range(0, 12)] + ['12+']

df['duration_bucket'] = pd.cut(df['duration_hours'], bins=bins, labels=labels, right=False)

# Step 3: Count videos in each bucket
histogram = df['duration_bucket'].value_counts().sort_index()

print(histogram)

# Step 4: Optional: plot it
histogram.plot(kind='bar')
plt.title('Number of Videos by Duration Bucket')
plt.xlabel('Duration (hours)')
plt.ylabel('Number of Videos')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# remove things over 4 hours (usually infinite loop videos)
df = df[df['duration_seconds'] <= 4 * 3600]

In [ ]:
# Assuming your parsed datetime is in 'watch_time_dt'
year = 2024

# Filter for rows where the year matches
df_year = df[df['watch_time_dt'].dt.year == year]

In [ ]:
len(df_year)

In [ ]:
json_stats = {}

In [ ]:
json_stats["year"] = year

In [ ]:
top_channels = (
    df_year['channel_name']
    .value_counts()
    .head(5)
    .to_dict()
)

In [ ]:
top_channels

In [ ]:
json_stats["top_channels"] = list(top_channels.keys())

In [ ]:
top_channels = (
    df_year
    .assign(minutes=df_year['duration_seconds'].fillna(0) / 60)
    .groupby('channel_name')['minutes']
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .to_dict()
)
top_channels

In [ ]:
# First get top channels' names
top_channel_names = (
    df_year
    .assign(minutes=df_year['duration_seconds'].fillna(0) / 60)
    .groupby('channel_name')['minutes']
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index.tolist()
)

# Now build a list of (channel_name, channel_link)
top_channels_with_links = (
    df_year
    .drop_duplicates(subset=['channel_name'])  # Remove duplicates first
    .set_index('channel_name')                 # Set channel_name as index
    .loc[top_channel_names]                    # Select only top channels
    [['channel_link']]                         # Only keep channel_link
    .reset_index()
    .values.tolist()
)


In [ ]:
top_channels_with_links

In [ ]:
json_stats["top_channels_links"] = [link[1] for link in top_channels_with_links]

In [ ]:
json_stats["top_channels_links"]

In [ ]:
import re
def extract_video_id(video_link):
    match = re.search(r'v=([^&]+)', video_link)
    return match.group(1) if match else None

In [ ]:
# Get top 5 videos by minutes watched
top_video_names = (
    df_year['video_name']
    .value_counts()
    .head(5)
    .index.tolist()
)

# Now fetch the video_link and construct thumbnail
top_videos_with_links_and_thumbs = []

for video_name in top_video_names:
    row = (
        df_year[df_year['video_name'] == video_name]
        .drop_duplicates(subset=['video_name'])  # Just in case
        .iloc[0]
    )
    video_link = row['video_link']
    video_id = extract_video_id(video_link)
    if video_id:
        thumbnail = f"https://i.ytimg.com/vi/{video_id}/mqdefault.jpg"
    else:
        thumbnail = "/api/placeholder/80/60"  # fallback if parsing failed

    top_videos_with_links_and_thumbs.append(
        (video_name, thumbnail, video_link)
    )


In [ ]:
top_videos_with_links_and_thumbs

In [ ]:
json_stats["top_videos_links"] = [link[2] for link in top_videos_with_links_and_thumbs]

In [ ]:
json_stats["top_videos_thumbs"] = [link[1] for link in top_videos_with_links_and_thumbs]

In [ ]:
top_videos = (
    df_year['video_name']
    .value_counts()
    .head(5)
    .to_dict()
)

In [ ]:
json_stats["top_videos"] = list(top_videos.keys())

In [ ]:
top_videos

In [ ]:
total_views = len(df_year)
total_views

In [ ]:
json_stats["total_views"] = total_views

In [ ]:
total_minutes = df_year['duration_seconds'].sum() // 60
total_minutes

In [ ]:
json_stats["total_hours"] = int(total_minutes // 60)

In [ ]:
json_stats["total_minutes"] = int(total_minutes - int(total_minutes // 60))

In [ ]:
top_day = (
    df_year['watch_time_dt']
    .dt.day_name()
    .value_counts()
    .idxmax()
)
top_day

In [ ]:
json_stats["top_day"] = top_day

In [ ]:
# Assuming you already filtered your df to a year like df_year
days_watched = df_year['watch_time_dt'].dt.date.nunique()
days_watched

In [ ]:
json_stats["total_days"] = days_watched

In [ ]:
average_minutes_per_day = total_minutes / days_watched if days_watched > 0 else 0

# average_minutes_per_day is already calculated
average_hours = int(average_minutes_per_day // 60)
average_minutes = int(average_minutes_per_day % 60)

print(f"🎬 Average YouTube watched per day: {average_hours}h {average_minutes}m")


In [ ]:
json_stats["average_hours"] = average_hours
json_stats["average_minutes"] = average_minutes

In [ ]:
# Group by date and sum minutes watched
minutes_per_day = (
    df_year
    .assign(minutes=df_year['duration_seconds'].fillna(0) / 60)
    .groupby(df_year['watch_time_dt'].dt.date)['minutes']
    .sum()
)

# Find top day
top_day = minutes_per_day.idxmax()
top_minutes = minutes_per_day.max()

# Convert top_day to pandas Timestamp to easily get day of week
top_day_dt = pd.to_datetime(top_day)
top_day_name = top_day_dt.day_name()

print(f"🏆 Top YouTube day: {top_day} ({top_day_name}) with {top_minutes:.2f} minutes watched!")


In [ ]:
json_stats["top_day_date_year"] = top_day.year

In [ ]:
json_stats["top_day_date_month"] = top_day.month

In [ ]:
json_stats["top_day_date_day"] = top_day.day

In [ ]:
json_stats["top_day_date_day_name"] = top_day_name

In [ ]:
json_stats["top_day_minutes"] = int(top_minutes)

In [ ]:
json_stats

In [ ]:
import json
with open("youtube-wrapped.json", "w") as f:
    f.write(json.dumps(json_stats))